# State level Assembly Constituency Mapper

This notebook maps each state's master list details to the attribute table of asn AC shapefile, which will then enable the user to create state level AC identifier maps. These maps, would then inherit the features present in the masterlist and will provide a concise undertsanding of the current stage of data availability, presence of missing constitutencies, presenc eof image file for certain constituencies etc. This exercise is an intermediate priocess to assess the data generation process and its progress. This notebook doesn't serve to provide any data wrangling abilities or wrnagle any existing raw or interim data files.


In [8]:
#importing necessary libraries
import pandas as pd
from pathlib import Path
from fuzzywuzzy import process

In [25]:
#defining directories
def dir_define(state:str):
    master_file_name="_".join([state,"masterlist.xlsx"])
    attr_file_name="_".join([state,"attr_table.csv"])
    dir_path=Path.cwd().parents[1]
    masterlist_path=dir_path.joinpath(f"data_docs/golden_states_masterlist/{master_file_name}")
    attr_table_path=dir_path.joinpath(f"data/external/attribute_tables/{attr_file_name}")

    mapped_attr_folder=dir_path.joinpath(f"data/interim/ac_mapped/{state}")

    if not mapped_attr_folder.exists():
        mapped_attr_folder.mkdir(parents=True)

    return masterlist_path, attr_table_path, mapped_attr_folder




In [47]:
def master_reader(state:str):

    masterlist_path=dir_define(state=state)[0]

    df=pd.read_excel(masterlist_path)

    df['state']=df['state'].str.lower().str.strip().str.replace(" ", "_")
    df['ac_name']=df['ac_name'].str.lower().str.strip().str.replace(" ", "_")
    print(df['ac_name'].nunique())

    return df

In [48]:
def attr_table_reader(state:str):
    attr_table_path=dir_define(state=state)[1]

    df=pd.read_csv(attr_table_path)
    shp_cols=df.columns
    shp_cols=[x.lower() for x in shp_cols]
    df.columns=shp_cols

    df['st_name']=df['st_name'].str.lower().str.strip().str.replace(" ", "_")
    df['ac_name']=df['ac_name'].str.lower().str.strip().str.replace(" ", "_")

    print(df['ac_name'].nunique())

    return df

In [49]:

def map_ac():
    master=master_reader(state='chhattisgarh')
    shp_file=attr_table_reader(state='chhattisgarh')

    # shp_file['id']=shp_file['st_name'] + shp_file['ac_name']
    # master['id']=master['state']+master['ac_name']

    result = [
        process.extractOne(i, shp_file['ac_name'])
        for i in master['ac_name']
    ]

    result = pd.DataFrame(result, columns=["match", "score", "id"])
    result=result[result['score']>=95]

    ac_dict=dict(zip(result['match'],master['ac_name']))

    print(result)

    shp_file['master_ac_name']=shp_file['ac_name'].map(ac_dict)

    # print(shp_file['master_ac_name'].isna().value_counts())

    final_file=dir_define(state='chhattisgarh')[2].joinpath('ac_mapped.csv')

    shp_file.to_csv(final_file, index=False)

    

map_ac()

d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


88
90
                  match  score  id
0   bharatpur-sonhat_(s     97   2
1          manendragarh    100   8
2           baikunthpur    100   5
3             premnagar    100   6
4              bhatgaon    100   3
..                  ...    ...  ..
84          bastar_(st)    100  84
85            jagdalpur    100  88
86       chitrakot_(st)    100  86
88         bijapur_(st)    100  85
89           konta_(st)    100  89

[78 rows x 3 columns]
